In [4]:
import geopandas as gpd
import pandas as pd
import fiona
from shapely.geometry import Polygon, LineString, Point
from geopandas import GeoSeries


In [20]:
wb_df = gpd.read_file("data/Humber.waterbodies.gpkg")
wb_df['wb_id'] = wb_df.index

/home/javad/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [21]:
br_df = gpd.read_file('data/Humber_Barriers.gpkg')

br_df.head()

,barrier_ty,geometry
0,grand,POINT (-57.25639 49.19823)
1,nhn,POINT (-57.03635 49.63076)
2,nhn,POINT (-58.08564 49.06965)
3,nhn,POINT (-57.81380 48.93041)
4,nhn,POINT (-57.07446 49.66377)


In [25]:
br_df = br_df.to_crs("EPSG:4326") # crs of waterbodies is 4326
br_df.head()

,barrier_ty,geometry
0,grand,POINT (-57.25639 49.19824)
1,nhn,POINT (-57.03634 49.63077)
2,nhn,POINT (-58.08563 49.06966)
3,nhn,POINT (-57.81380 48.93042)
4,nhn,POINT (-57.07446 49.66378)


## Intersection between waterbodies and barriers
Create connection between barriers and waterbodies

In [27]:
br_wb_intesect_df = gpd.overlay(br_df, wb_df, how='intersection', make_valid=True, keep_geom_type=True)

In [28]:
br_wb_intesect_df.head()

,barrier_ty,region_id,area,definition,wb_id,geometry
0,grand,Humber,1.257443e+08,-1,7884,POINT (-57.25639 49.19824)
1,nhn,Humber,1.257443e+08,-1,7884,POINT (-57.39176 49.48927)
2,nhn,Humber,1.257443e+08,-1,7884,POINT (-57.39299 49.48980)
3,nlprov,Humber,1.257443e+08,-1,7884,POINT (-57.09722 49.51945)
4,npdp,Humber,1.257443e+08,-1,7884,POINT (-57.43661 49.17022)


## Segments


is there an intersection between barriers and segments?

In [41]:
sg_df = gpd.read_file("data/Humber.network.gpkg").set_crs("EPSG:4326")
sg_df.head()

/home/javad/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


,region_id,type,rank,length,name,nameid,from_node_id,to_node_id,geometry
0,Humber,Inferred,Primary,52.404344,None,None,26762,26764,"LINESTRING (-57.96237 49.25319, -57.96233 49.2..."
1,Humber,Inferred,Primary,38.001086,None,None,8150,8147,"LINESTRING (-57.08230 49.42386, -57.08234 49.4..."
2,Humber,Inferred,Primary,41.694741,None,None,8151,8150,"LINESTRING (-57.08232 49.42422, -57.08235 49.4..."
3,Humber,Bank,Primary,21.196205,None,None,8148,8150,"LINESTRING (-57.08210 49.42372, -57.08220 49.4..."
4,Humber,Inferred,Primary,14.079574,None,None,51032,51029,"LINESTRING (-57.50818 49.73562, -57.50818 49.7..."


In [42]:
br_sg_intesect_df = gpd.overlay(br_df, sg_df, how='intersection', make_valid=True, keep_geom_type=True)

In [43]:
br_sg_intesect_df

,to_node_id,rank,type,length,region_id,nameid,barrier_ty,name,from_node_id,geometry


The empty results show that there's no intersection.


### Finding closest segment

In [71]:
import itertools
from operator import itemgetter

import geopandas as gpd
import numpy as np
import pandas as pd

from scipy.spatial import cKDTree
from shapely.geometry import Point, LineString


def ckdnearest_line(gdfA, gdfB, gdfB_cols=['name']):
    """ A is point and B is Linestring """
    # resetting the index of gdfA and gdfB here.
    gdfA = gdfA.reset_index(drop=True)
    gdfB = gdfB.reset_index(drop=True)
    A = np.concatenate(
        [np.array(geom.coords) for geom in gdfA.geometry.to_list()])
    B = [np.array(geom.coords) for geom in gdfB.geometry.to_list()]
    B_ix = tuple(itertools.chain.from_iterable(
        [itertools.repeat(i, x) for i, x in enumerate(list(map(len, B)))]))
    B = np.concatenate(B)
    ckd_tree = cKDTree(B)
    dist, idx = ckd_tree.query(A, k=1)
    idx = itemgetter(*idx)(B_ix)
    gdf = pd.concat(
        [gdfA, gdfB.loc[idx, gdfB_cols].reset_index(drop=True),
         pd.Series(dist, name='dist')], axis=1)
    return gdf



In [46]:
sg_cols = ['region_id', 'type', 'rank', 'length', 'name', 'nameid', 'from_node_id',
       'to_node_id'] # to keep, modify this later

In [72]:
br_sg_nearest_df = ckdnearest_line(br_df, sg_df, sg_cols)

## Find closest waterbody to a barrier

In [74]:

def ckdnearest_poly(gdfA, gdfB, gdfB_cols):
    """ A is point and B is Polygon """
    gdfA = gdfA.reset_index(drop=True)
    gdfB = gdfB.reset_index(drop=True)
    A = np.concatenate(
        [np.array(geom.coords) for geom in gdfA.geometry.to_list()])
    B = [np.array(geom.coords) for geom in gdfB.exterior]
    B_ix = tuple(itertools.chain.from_iterable(
        [itertools.repeat(i, x) for i, x in enumerate(list(map(len, B)))]))
    B = np.concatenate(B)
    ckd_tree = cKDTree(B)
    dist, idx = ckd_tree.query(A, k=1)
    idx = itemgetter(*idx)(B_ix)
    gdf = pd.concat(
        [gdfA, gdfB.loc[idx, gdfB_cols].reset_index(drop=True),
         pd.Series(dist, name='dist')], axis=1)
    return gdf



In [75]:
br_wb_nearest_df = ckdnearest_poly(br_df, wb_df, ['wb_id'])

In [76]:
br_wb_nearest_df.head()

,barrier_ty,geometry,wb_id,dist
0,grand,POINT (-57.25639 49.19824),7884,0.000011
1,nhn,POINT (-57.03634 49.63077),3217,0.000011
2,nhn,POINT (-58.08563 49.06966),4989,0.000011
3,nhn,POINT (-57.81380 48.93042),2467,0.000011
4,nhn,POINT (-57.07446 49.66378),4007,0.000011


##  Summary:

- Connections between barrier--waterbodies and barrier--segments are created.
- Use the connections to create csv file for Neo4J

## Next step:

- Prepare sites' data and convert from excel to geo format
- Find the closest location (segment / water body / barrier) to each site
- Define a threshold for "being at/in" a location
- create a relationship between site and these three nodes as `located in/at`

In the point, geological relationships are done.   **Note**: junction and segments are connected by the `overlay` function.